# Task C1
In the beginning we read the CSV files as dictionary and then create a TCP streaming server on localhost:9999 to send a batch of documents every second to the Spark Client.

We decided to add timestamp from gmtime, strftime library to each record for further validation of the relevant records. 

We also added our own delimiter("||") to distinguish between each record and at the end of each batch there is a "\n" for receiving the record at Spark Client.


In [1]:
import socket
import csv
import json
from time import sleep
from time import gmtime, strftime
import csv
import json

def read_csv(file, format):
    """
    Read CSV files and load them into memory as a dictionary.

    Args:
        csvFile -- Location and the name of the csvFile( i.e C:/FireData.csv or FireData.csv which is
        located in the same folder as the *.ipynb file)
        
        format -- format of the file 

    Returns:
        csv_rows -- loaded data ready for processing
    
    """
    csv_rows = []
    with open(file) as csvfile:
        # read the csv file by DictReader 
        reader = csv.DictReader(csvfile)
        title = reader.fieldnames
        for row in reader:
            # adding relevant values to relevant keys and strip any space from both
            # add the dictionary to csv_rows
            csv_rows.extend([{title[i].strip():row[title[i]].strip() for i in range(len(title))}])
    return csv_rows


fireData = read_csv('FireData-Part2.csv', csv)
climateData = read_csv('ClimateData-Part2.csv', csv)

"""
#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*
"""

"""
TCP Stream Generator Server
"""
host = 'localhost'
port = 9999

# Create an INET, STREAMing socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Bind the socket to a public host,and a port
s.bind((host, port))
# Become a server socket
s.listen(1)
while True:
    print('\nListening for a client at',host , port)
    # Accept the connection from everyone 
    conn, addr = s.accept()
    print('\nConnected by', addr)
    try:
        print('\nReading file...\n')
        for c in climateData:
            # Creating a time stamp
            dateTime = strftime('%Y-%m-%d@%H-%M-%S')
            # Adding time stamp to the ClimateData
            c['DateTime'] = dateTime
            fire_temp = []
            for i in range(8):
                if len(fireData) >= 8:
                    # Remove the item , and return it
                    fire_item = fireData.pop()
                    # Adding the time stamp
                    fire_item['DateTime'] = dateTime
                    # Serialize records as a string
                    fire_temp.append(json.dumps(fire_item))
                else:
                    for x in fireData[:len(fireData)]:
                        # Adding the time stamp
                        x['DateTime'] = dateTime
                        # Serialize records as a string
                        fire_temp.append(json.dumps(x))
                    break
            final_list = []
            # Serialize Climate Data record as a string
            final_list.append(json.dumps(c))
            final_list += fire_temp
            # Adding delimiter
            line = '||'.join(final_list)
            # Add new line
            line += '\n'
            # Encode it to Bytes
            out = line.encode('utf-8')
            print('Sending line',line)
            # Send the batch towards the Client
            conn.send(out)
            # Sleep for one second (Pause)
            sleep(1)
            print('End Of Stream.')
        
    except socket.error:
        # Catching any error for socket
        print ('Error Occured.\n\nClient disconnected.\n')
conn.close()


Listening for a client at localhost 9999

Connected by ('127.0.0.1', 53070)

Reading file...

Sending line {"DataType": "cdata", "Station": "948700", "Air Temperature(Celcius)": "19", "Relative Humidity": "56.8", "WindSpeed  (knots)": "7.9", "Max Wind Speed": "11.1", "MAX": "72.0*", "MIN": "61.9*", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-42-27"}||{"DataType": "fdata", "Latitude": "-34.3735", "Longitude": "141.6604", "Surface Temperature (kelvin)": "332.2", "Power": "16.5", "Confidence": "85", "Surface Temperature (Celcius)": "59", "DateTime": "2018-05-21@14-42-27"}||{"DataType": "fdata", "Latitude": "-34.3795", "Longitude": "141.6331", "Surface Temperature (kelvin)": "335.4", "Power": "21.7", "Confidence": "87", "Surface Temperature (Celcius)": "62", "DateTime": "2018-05-21@14-42-27"}||{"DataType": "fdata", "Latitude": "-37.7752", "Longitude": "141.9086", "Surface Temperature (kelvin)": "337.8", "Power": "30", "Confidence": "88", "Surface Temperature (Celcius)": "64", "Da

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948700", "Air Temperature(Celcius)": "24", "Relative Humidity": "62.3", "WindSpeed  (knots)": "7", "Max Wind Speed": "13", "MAX": "85.5*", "MIN": "65.7*", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-42-31"}||{"DataType": "fdata", "Latitude": "-34.357", "Longitude": "141.5361", "Surface Temperature (kelvin)": "340.5", "Power": "28.9", "Confidence": "90", "Surface Temperature (Celcius)": "67", "DateTime": "2018-05-21@14-42-31"}||{"DataType": "fdata", "Latitude": "-37.4098", "Longitude": "147.0489", "Surface Temperature (kelvin)": "319.1", "Power": "15.5", "Confidence": "70", "Surface Temperature (Celcius)": "45", "DateTime": "2018-05-21@14-42-31"}||{"DataType": "fdata", "Latitude": "-37.0249", "Longitude": "148.1561", "Surface Temperature (kelvin)": "322.6", "Power": "23.5", "Confidence": "75", "Surface Temperature (Celcius)": "49", "DateTime": "2018-05-21@14-42-31"}||{"DataType": "fdata", "Latitude": "-37.002", "Long

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948700", "Air Temperature(Celcius)": "19", "Relative Humidity": "56.3", "WindSpeed  (knots)": "5.8", "Max Wind Speed": "8.9", "MAX": "73.6*", "MIN": "60.6*", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-42-35"}||{"DataType": "fdata", "Latitude": "-37.4523", "Longitude": "147.0175", "Surface Temperature (kelvin)": "339.7", "Power": "46.6", "Confidence": "90", "Surface Temperature (Celcius)": "66", "DateTime": "2018-05-21@14-42-35"}||{"DataType": "fdata", "Latitude": "-37.3982", "Longitude": "147.0001", "Surface Temperature (kelvin)": "331.6", "Power": "25.6", "Confidence": "64", "Surface Temperature (Celcius)": "58", "DateTime": "2018-05-21@14-42-35"}||{"DataType": "fdata", "Latitude": "-37.9257", "Longitude": "143.4891", "Surface Temperature (kelvin)": "327.9", "Power": "12.3", "Confidence": "63", "Surface Temperature (Celcius)": "54", "DateTime": "2018-05-21@14-42-35"}||{"DataType": "fdata", "Latitude": "-37.6622", 

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948700", "Air Temperature(Celcius)": "19", "Relative Humidity": "54.1", "WindSpeed  (knots)": "11.2", "Max Wind Speed": "18.1", "MAX": "81.0*", "MIN": "60.8*", "Precipitation": "0.31G", "DateTime": "2018-05-21@14-42-39"}||{"DataType": "fdata", "Latitude": "-37.8289", "Longitude": "145.9315", "Surface Temperature (kelvin)": "322.3", "Power": "23.7", "Confidence": "75", "Surface Temperature (Celcius)": "49", "DateTime": "2018-05-21@14-42-39"}||{"DataType": "fdata", "Latitude": "-38.1106", "Longitude": "143.7314", "Surface Temperature (kelvin)": "326.5", "Power": "21.5", "Confidence": "80", "Surface Temperature (Celcius)": "53", "DateTime": "2018-05-21@14-42-39"}||{"DataType": "fdata", "Latitude": "-38.1131", "Longitude": "143.7173", "Surface Temperature (kelvin)": "317.5", "Power": "10.2", "Confidence": "68", "Surface Temperature (Celcius)": "44", "DateTime": "2018-05-21@14-42-39"}||{"DataType": "fdata", "Latitude": "-38.018",

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948700", "Air Temperature(Celcius)": "24", "Relative Humidity": "53.4", "WindSpeed  (knots)": "9.9", "Max Wind Speed": "15.9", "MAX": "95.0*", "MIN": "62.8*", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-42-43"}||{"DataType": "fdata", "Latitude": "-36.4483", "Longitude": "142.2303", "Surface Temperature (kelvin)": "322.9", "Power": "14.5", "Confidence": "76", "Surface Temperature (Celcius)": "49", "DateTime": "2018-05-21@14-42-43"}||{"DataType": "fdata", "Latitude": "-36.4811", "Longitude": "142.2606", "Surface Temperature (kelvin)": "324.1", "Power": "15.3", "Confidence": "77", "Surface Temperature (Celcius)": "50", "DateTime": "2018-05-21@14-42-43"}||{"DataType": "fdata", "Latitude": "-37.9526", "Longitude": "143.597", "Surface Temperature (kelvin)": "380.9", "Power": "158.5", "Confidence": "100", "Surface Temperature (Celcius)": "107", "DateTime": "2018-05-21@14-42-43"}||{"DataType": "fdata", "Latitude": "-37.9452

...

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948730", "Air Temperature(Celcius)": "25", "Relative Humidity": "60.9", "WindSpeed  (knots)": "6.3", "Max Wind Speed": "8.9", "MAX": "86.2*", "MIN": "60.3", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-53-36"}||{"DataType": "fdata", "Latitude": "-37.966", "Longitude": "145.051", "Surface Temperature (kelvin)": "341.8", "Power": "26.7", "Confidence": "78", "Surface Temperature (Celcius)": "68", "DateTime": "2018-05-21@14-53-36"}||{"DataType": "fdata", "Latitude": "-35.541", "Longitude": "143.311", "Surface Temperature (kelvin)": "336.3", "Power": "62", "Confidence": "82", "Surface Temperature (Celcius)": "63", "DateTime": "2018-05-21@14-53-36"}||{"DataType": "fdata", "Latitude": "-35.554", "Longitude": "143.307", "Surface Temperature (kelvin)": "326.8", "Power": "23.8", "Confidence": "67", "Surface Temperature (Celcius)": "53", "DateTime": "2018-05-21@14-53-36"}||{"DataType": "fdata", "Latitude": "-35.543", "Longitude

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948730", "Air Temperature(Celcius)": "19", "Relative Humidity": "53.9", "WindSpeed  (knots)": "8.3", "Max Wind Speed": "14", "MAX": "78.1*", "MIN": "57.2", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-53-41"}||{"DataType": "fdata", "Latitude": "-37.966", "Longitude": "145.051", "Surface Temperature (kelvin)": "341.8", "Power": "26.7", "Confidence": "78", "Surface Temperature (Celcius)": "68", "DateTime": "2018-05-21@14-53-41"}||{"DataType": "fdata", "Latitude": "-35.541", "Longitude": "143.311", "Surface Temperature (kelvin)": "336.3", "Power": "62", "Confidence": "82", "Surface Temperature (Celcius)": "63", "DateTime": "2018-05-21@14-53-41"}||{"DataType": "fdata", "Latitude": "-35.554", "Longitude": "143.307", "Surface Temperature (kelvin)": "326.8", "Power": "23.8", "Confidence": "67", "Surface Temperature (Celcius)": "53", "DateTime": "2018-05-21@14-53-41"}||{"DataType": "fdata", "Latitude": "-35.543", "Longitude"

End Of Stream.
Sending line {"DataType": "cdata", "Station": "948730", "Air Temperature(Celcius)": "21", "Relative Humidity": "61.1", "WindSpeed  (knots)": "6.6", "Max Wind Speed": "11.1", "MAX": "90.0*", "MIN": "64.8*", "Precipitation": "0.00I", "DateTime": "2018-05-21@14-53-46"}||{"DataType": "fdata", "Latitude": "-37.966", "Longitude": "145.051", "Surface Temperature (kelvin)": "341.8", "Power": "26.7", "Confidence": "78", "Surface Temperature (Celcius)": "68", "DateTime": "2018-05-21@14-53-46"}||{"DataType": "fdata", "Latitude": "-35.541", "Longitude": "143.311", "Surface Temperature (kelvin)": "336.3", "Power": "62", "Confidence": "82", "Surface Temperature (Celcius)": "63", "DateTime": "2018-05-21@14-53-46"}||{"DataType": "fdata", "Latitude": "-35.554", "Longitude": "143.307", "Surface Temperature (kelvin)": "326.8", "Power": "23.8", "Confidence": "67", "Surface Temperature (Celcius)": "53", "DateTime": "2018-05-21@14-53-46"}||{"DataType": "fdata", "Latitude": "-35.543", "Longitu

End Of Stream.

Listening for a client at localhost 9999


KeyboardInterrupt: 